### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('Datasets/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('Datasets/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
print(movies.head())
print('\n')
print(reviews.head())

  movie_id                                              movie  \
0  0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1  0000010               La sortie des usines LumiÃ¨re (1895)   
2  0000012                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4  0000091                         Le manoir du diable (1896)   

               genre  
0  Documentary|Short  
1  Documentary|Short  
2  Documentary|Short  
3                NaN  
4       Short|Horror  


  user_id  movie_id  rating   timestamp
0       1   0114508       8  1381006850
1       2   0358273       9  1579057827
2       2  10039344       5  1578603053
3       2   6751668       9  1578955697
4       2   7131622       8  1579559244


In [3]:
# cell for work
movies.shape[0]

35112

In [4]:
# more work
reviews.shape[0]

850515

In [5]:
# and still more
genres = []
for gen in movies['genre']:
    try:
        genres.extend(gen.split('|'))
    except:
        pass
genres = set(genres)
len(genres)

28

In [6]:
# and even more - open additional cells as necessary
reviews.user_id.nunique()

65964

In [7]:
reviews.rating.isnull().sum()

0

In [8]:
reviews.rating.describe()

count    850515.000000
mean          7.314708
std           1.854076
min           0.000000
25%           6.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

In [9]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 65964
b = 10
c = 7
d = 35112
e = 15
f = 0
g = 4
h = 850515
i = 28

dict_sol1 = {
'The number of movies in the dataset': d,
'The number of ratings in the dataset': h,
'The number of different genres': i,
'The number of unique users in the dataset': a,
'The number missing ratings in the reviews dataset': f,
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': f,
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp
* Create month/day/year columns from timestamp as dummies

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [10]:
# cell for work
create_date = lambda x: x[-5:-1] if x[-1] == ')' else np.nan
movies['date'] = movies['movie'].apply(create_date)

In [11]:
# and another
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines LumiÃ¨re (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [12]:
# and a few more below, which you can use as necessary
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
    
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['date'].apply(add_movie_year)
    
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines LumiÃ¨re (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [13]:
def add_genre(val):
    try:
        if genre in val.split('|'):
            return 1
        else:
            return 0
    except AttributeError:
        return 0
    
for genre in genres:
    movies[genre] = movies['genre'].apply(add_genre)

movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Mystery,Crime,Biography,...,Adventure,Film-Noir,Talk-Show,Fantasy,Romance,Sci-Fi,Family,Musical,War,Short
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0000010,La sortie des usines LumiÃ¨re (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
movies.iloc[1]

movie_id                                    0000010
movie          La sortie des usines LumiÃ¨re (1895)
genre                             Documentary|Short
date                                           1895
1800's                                            1
1900's                                            0
2000's                                            0
Mystery                                           0
Crime                                             0
Biography                                         0
Game-Show                                         0
Sport                                             0
Documentary                                       1
Western                                           0
Thriller                                          0
News                                              0
Comedy                                            0
History                                           0
Animation                                         0
Reality-TV  

In [15]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0358273,9,1579057827
2,2,10039344,5,1578603053
3,2,6751668,9,1578955697
4,2,7131622,8,1579559244


In [16]:
from datetime import datetime

change_timestamp = lambda x: datetime.fromtimestamp(int(x))

reviews['date_time'] = reviews['timestamp'].apply(change_timestamp)

reviews.head()

,user_id,movie_id,rating,timestamp,date_time
0,1,0114508,8,1381006850,2013-10-06 02:30:50
1,2,0358273,9,1579057827,2020-01-15 08:40:27
2,2,10039344,5,1578603053,2020-01-10 02:20:53
3,2,6751668,9,1578955697,2020-01-14 04:18:17
4,2,7131622,8,1579559244,2020-01-21 03:57:24


In [17]:
reviews.to_csv('./Datasets/reviews_clean.csv')
movies.to_csv('./Datasets/movies_clean.csv')